In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x673a0d0


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [5]:
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")

True

In [6]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

# ROOT.OOTools.init(250, mW, mZ)
ROOT.OOTools.init(190, mW, mZ)

In [7]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [8]:
%%cpp
using namespace ROOT::VecOps;

In [9]:
%%cpp
Double_t d_sigma(Int_t n_dim, Double_t *args)
{
    double co = args[0] * 2. - 1.;
    double co1 = args[1] * 2. - 1.;
    double co2 = args[2] * 2. - 1.;
    double ph1 = (args[3] * 2. - 1.) * ROOT::Math::Pi();
    double ph2 = (args[4] * 2. - 1.) * ROOT::Math::Pi();

    return OOTools::proba_sm(co, co1, co2, ph1, ph2);
}

In [10]:
%%cpp
TRandom3 rand{};
rand.SetSeed(1234);
TFoam foam ("foam");
foam.SetkDim(5);
foam.SetnCells(100);
foam.SetnSampl(500);
foam.SetPseRan(&rand);
foam.SetRhoInt(d_sigma);
foam.SetMaxWtRej(1.1);
foam.Initialize();

FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
F                                                                              F
F                   ****************************************                   F
F                    ******      TFoam::Initialize    ******                   F
F                   ****************************************                   F
F                                                       foam                   F
F    Version =           1.02M   =                   Release date:  2005.04.10 F
F       kDim =          5 =  Dimension of the hyper-cubical space              F
F     nCells =        100 =  Requested number of Cells (half of them active)   F
F     nSampl =        500 =  No of MC events in exploration of a cell          F
F       nBin =          8 =  No of bins in histograms, MC exploration of cell  F
F   EvPerBin =         25 =  Maximum No effective_events/bin, MC exploration   F
F   OptDrive =          2 = 

In [11]:
# n_points = 5000000
n_points = 100000

df = ROOT.RDataFrame(n_points)

In [12]:
df = df.Define("mc_record", "Double_t pos[5]; Double_t w = foam.MCgenerate(pos); return std::make_tuple(w, pos[0], pos[1], pos[2], pos[3], pos[4])")
df = df.Define("mc_weight", "std::get<0>(mc_record)")
df = df.Define("co", "std::get<1>(mc_record) * 2. - 1.")
df = df.Define("co1", "std::get<2>(mc_record) * 2. - 1.")
df = df.Define("co2", "std::get<3>(mc_record) * 2. - 1.")
df = df.Define("ph1", "(std::get<4>(mc_record) * 2. - 1.) * ROOT::Math::Pi()")
df = df.Define("ph2", "(std::get<5>(mc_record) * 2. - 1.) * ROOT::Math::Pi()")

In [13]:
for var in [f"O_fg{i}" for i in range(1,8)] + [f"O_fz{i}" for i in range(1,8)]:
    df = df.Define(var, f"OOTools::{var}(co, co1, co2, ph1, ph2)")

In [14]:
pi = ROOT.Math.Pi()

# h_co  = df.Histo1D(("", ";co", 25, -1., 1.), "co", "w_proba")
# h_co1 = df.Histo1D(("", ";co1", 25, -1., 1.), "co1", "w_proba")
# h_co2 = df.Histo1D(("", ";co2", 25, -1., 1.), "co2", "w_proba")
# h_ph1 = df.Histo1D(("", ";ph1", 25, -pi, pi), "ph1", "w_proba")
# h_ph2 = df.Histo1D(("", ";ph2", 25, -pi, pi), "ph2", "w_proba")

h_co  = df.Histo1D(("", ";co", 25, -1., 1.), "co")
h_co1 = df.Histo1D(("", ";co1", 25, -1., 1.), "co1")
h_co2 = df.Histo1D(("", ";co2", 25, -1., 1.), "co2")
h_ph1 = df.Histo1D(("", ";ph1", 25, -pi, pi), "ph1")
h_ph2 = df.Histo1D(("", ";ph2", 25, -pi, pi), "ph2")


h_weight = df.Histo1D("mc_weight")

h_fg1 = df.Histo1D(("", ";O_fg1", 100, -1.5, 1.5), "O_fg1")
h_fg2 = df.Histo1D(("", ";O_fg2", 100, -1.5, 1.5), "O_fg2")
h_fg3 = df.Histo1D(("", ";O_fg3", 100, -1.5, 1.5), "O_fg3")
h_fg4 = df.Histo1D(("", ";O_fg4", 100, -1.5, 1.5), "O_fg4")
h_fg5 = df.Histo1D(("", ";O_fg5", 100, -1.5, 1.5), "O_fg5")

h_fz1 = df.Histo1D(("", ";O_fz1", 100, -1.5, 1.5), "O_fg1")
h_fz2 = df.Histo1D(("", ";O_fz2", 100, -1.5, 1.5), "O_fg2")
h_fz3 = df.Histo1D(("", ";O_fz3", 100, -1.5, 1.5), "O_fg3")
h_fz4 = df.Histo1D(("", ";O_fz4", 100, -1.5, 1.5), "O_fg4")
h_fz5 = df.Histo1D(("", ";O_fz5", 100, -1.5, 1.5), "O_fg5")

In [15]:
c_co = ROOT.TCanvas()
# p_co.SetMinimum(0)
# p_co.Draw()
h_co.SetMinimum(0)
h_co.Draw()
c_co.Draw()

c_co1 = ROOT.TCanvas()
# p_co1.SetMinimum(0)
# p_co1.Draw()
h_co1.SetMinimum(0)
h_co1.Draw()
c_co1.Draw()

c_ph1 = ROOT.TCanvas()
# p_ph1.SetMinimum(0)
# p_ph1.Draw()
h_ph1.SetMinimum(0)
h_ph1.Draw()
c_ph1.Draw()

c_co2 = ROOT.TCanvas()
# p_co2.SetMinimum(0)
# p_co2.Draw()
h_co2.SetMinimum(0)
h_co2.Draw()
c_co2.Draw()

c_ph2 = ROOT.TCanvas()
# p_ph2.SetMinimum(0)
# p_ph2.Draw()
h_ph2.SetMinimum(0)
h_ph2.Draw()
c_ph2.Draw()

c_weight = ROOT.TCanvas()
h_weight.SetMinimum(0)
h_weight.Draw()
c_weight.Draw()


In [16]:

c_fg1 = ROOT.TCanvas()
h_fg1.SetMinimum(0)
h_fg1.Draw()
c_fg1.Draw()

c_fg2 = ROOT.TCanvas()
h_fg2.SetMinimum(0)
h_fg2.Draw()
c_fg2.Draw()

c_fg3 = ROOT.TCanvas()
h_fg3.SetMinimum(0)
h_fg3.Draw()
c_fg3.Draw()

c_fg4 = ROOT.TCanvas()
h_fg4.SetMinimum(0)
h_fg4.Draw()
c_fg4.Draw()

c_fg5 = ROOT.TCanvas()
h_fg5.SetMinimum(0)
h_fg5.Draw()
c_fg5.Draw()

c_fz1 = ROOT.TCanvas()
h_fz1.SetMinimum(0)
h_fz1.Draw()
c_fz1.Draw()

c_fz2 = ROOT.TCanvas()
h_fz2.SetMinimum(0)
h_fz2.Draw()
c_fz2.Draw()

c_fz3 = ROOT.TCanvas()
h_fz3.SetMinimum(0)
h_fz3.Draw()
c_fz3.Draw()

c_fz4 = ROOT.TCanvas()
h_fz4.SetMinimum(0)
h_fz4.Draw()
c_fz4.Draw()

c_fz5 = ROOT.TCanvas()
h_fz5.SetMinimum(0)
h_fz5.Draw()
c_fz5.Draw()

In [17]:
%%cpp
foam.PrintCells();

Cell[0]={ 
 Status= 0, Volume= 1, TrueInteg= 26.474305, DriveInteg= 74.243658, PrimInteg= 210.36753,
 Xdiv= 0.625, Best= 0, Parent=  {-1}  Daught0= {1}  Daught1= {2} 
   Posi= (           0,           0,           0,           0,           0),
   Size= (           1,           1,           1,           1,           1),
}
Cell[1]={ 
 Status= 0, Volume= 0.625, TrueInteg= 7.7011434, DriveInteg= 27.130823, PrimInteg= 62.40137,
 Xdiv= 0.375, Best= 4, Parent=  {0}  Daught0= {7}  Daught1= {8} 
   Posi= (           0,           0,           0,           0,           0),
   Size= (       0.625,           1,           1,           1,           1),
}
Cell[2]={ 
 Status= 0, Volume= 0.375, TrueInteg= 18.773162, DriveInteg= 47.112835, PrimInteg= 161.42841,
 Xdiv= 0.625, Best= 1, Parent=  {0}  Daught0= {3}  Daught1= {4} 
   Posi= (       0.625,           0,           0,           0,           0),
   Size= (       0.375,           1,           1,           1,           1),
}
Cell[3]={ 
 Status= 0, Vol